In [1]:
import pandas as pd
import numpy as np
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

In [2]:
# CSV 파일 읽기
file_path = '../data/justwatch_second_1090.xlsx'
df = pd.read_excel(file_path)

# 열 이름 확인
print(df.columns)

Index(['Unnamed: 0', 'title', 'original_title', 'year', 'season_episode',
       'runtime', 'genre', 'age_rating', 'Production country', 'IMDb_title',
       'IMDb_URL'],
      dtype='object')


In [3]:
# URL 리스트 추출 
urls = df['IMDb_URL'].tolist()
urls

['https://www.imdb.com/title/tt10919420/',
 'https://www.imdb.com/title/tt7605396/',
 'https://www.imdb.com/title/tt12079212/',
 'https://www.imdb.com/title/tt12809988/',
 'https://www.imdb.com/title/tt11612120/',
 'https://www.imdb.com/title/tt17491088/',
 'https://www.imdb.com/title/tt6470478/',
 'https://www.imdb.com/title/tt1520211/',
 'https://www.imdb.com/title/tt1526318/',
 'https://www.imdb.com/title/tt3107288/',
 'https://www.imdb.com/title/tt7622902/',
 'https://www.imdb.com/title/tt8740790/',
 'https://www.imdb.com/title/tt1632701/',
 'https://www.imdb.com/title/tt5753856/',
 'https://www.imdb.com/title/tt3006802/',
 'https://www.imdb.com/title/tt0873992/',
 'https://www.imdb.com/title/tt13068006/',
 'https://www.imdb.com/title/tt0903747/',
 'https://www.imdb.com/title/tt8824648/',
 'https://www.imdb.com/title/tt2306299/',
 'https://www.imdb.com/title/tt22352854/',
 'https://www.imdb.com/title/tt5290382/',
 'https://www.imdb.com/title/tt2661044/',
 'https://www.imdb.com/titl

In [4]:
len(urls)

1090

In [5]:
df['title']

0                  Ojingeo Geim
1                      12인의 심판자
2                        희생자 게임
3            스위트 투스: 사슴뿔을 가진 소년
4                    Sweet Home
                 ...           
1085                         달러
1086                      러브 나우
1087                     탕탕 라이브
1088                       대송궁사
1089    엘리트들, 못다 한 이야기: 나디아 구스만
Name: title, Length: 1090, dtype: object

In [6]:
df['original_title']

0                                 Squid Game
1                                 The Twelve
2                                      誰是被害者
3                                Sweet Tooth
4                                        NaN
                        ...                 
1085                                  Dollar
1086                                   真愛趁現在
1087                                糖糖Online
1088                                    大宋宫词
1089    Elite Histórias Breves: Nadia Guzmán
Name: original_title, Length: 1090, dtype: object

In [ ]:
#2
# 크롤링 결과를 저장할 리스트
data = []

# Selenium 웹 드라이버 설정
driver = webdriver.Chrome()

# XLSX 파일 로드
original_xlsx_path = '../data/justwatch_second_1090.xlsx'
original_df = pd.read_excel(original_xlsx_path)

# URL과 제목 리스트 추출
urls_titles = original_df[['IMDb_URL', 'title']]

for index, row in urls_titles.iterrows():
    url = row['IMDb_URL']
    title = row['title']
    try:
        # cast & crew 페이지 URL 생성
        fullcredits_url = url + 'fullcredits/?ref_=tt_ql_1'
        
        # cast & crew 페이지로 이동
        driver.get(fullcredits_url)

        # 페이지가 로드될 때까지 기다림
        WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, 'h1')))

        # 페이지 소스 가져오기
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')

        # 감독 정보 추출
        directors = []
        director_elements = soup.select('table.simpleCreditsTable:nth-of-type(1) a')
        for elem in director_elements:
            directors.append(elem.text.strip())
        
        # 작가 정보 추출
        writers = []
        writer_elements = soup.select('table.simpleCreditsTable:nth-of-type(2) a')
        for elem in writer_elements:
            writers.append(elem.text.strip())

        # 배우 정보 추출 (10명까지)
        actors = []
        actor_elements = soup.select('table.cast_list tr .primary_photo + td a')
        for elem in actor_elements[:10]:
            actors.append(elem.text.strip())

        # 결과를 리스트에 추가
        data.append({
            'IMDb_URL': url,
            'title': title,
            'fullcredits_url': fullcredits_url,
            'director': ', '.join(directors),
            'writer': ', '.join(writers),
            'actor': ', '.join(actors)
        })
        
    except Exception as e:
        print(f"Failed to process URL {url}: {e}")
        data.append({
            'IMDb_URL': url,
            'title': title,
            'fullcredits_url': fullcredits_url,
            'director': None,
            'writer': None,
            'actor': None
        })

# 드라이버 종료
driver.quit()

In [ ]:
len(data)

In [ ]:
# 데이터프레임으로 변환
result_df = pd.DataFrame(data)
result_df

In [ ]:
# 두 DataFrame 병합
merged_df = pd.merge(original_df, result_df, on=['IMDb_URL', 'title'], how='left')

# 중복된 감독과 작가 정보 제거
merged_df['director'] = merged_df['director'].apply(lambda x: ', '.join(sorted(set(x.split(', ')))) if pd.notnull(x) else x)
merged_df['writer'] = merged_df['writer'].apply(lambda x: ', '.join(sorted(set(x.split(', ')))) if pd.notnull(x) else x)

In [ ]:
# 최종 데이터프레임의 행 수 확인
print("Final row count:", merged_df.shape[0])

In [ ]:
merged_df.info()

In [ ]:
merged_df[merged_df['director'].isna()]

In [ ]:
import numpy as np

# None 값을 NaN으로 변경
merged_df['director'].replace({None: np.nan}, inplace=True)
merged_df['writer'].replace({None: np.nan}, inplace=True)
merged_df['actor'].replace({None: np.nan}, inplace=True)

# 변경 후 확인
print(merged_df[['director', 'writer', 'actor']].isna().sum())

In [ ]:
merged_df[merged_df['writer'].isna()]

In [ ]:
merged_df

In [ ]:
# 결과를 CSV 파일로 저장
csv_path = '../data/full_imdb_director_actor.csv'
merged_df.to_csv(csv_path, index=False, encoding='utf-8')

# 결과를 XLSX 파일로 저장
xlsx_path = '../data/full_imdb_director_actor.xlsx'
merged_df.to_excel(xlsx_path, index=False)

In [ ]:
# 삭제할 컬럼 목록
columns_to_drop = ['season_episode', 'runtime', 'genre', 'age_rating', 'Production country', 'IMDb_title', 'IMDb_URL']

# 특정 컬럼 제거
merged_df.drop(columns=columns_to_drop, inplace=True)

# 결과 확인
merged_df.tail()

In [ ]:
# 결과 확인
merged_df.tail()

In [ ]:
# 결과를 CSV 파일로 저장
csv_path = '../data/imdb_crew_cast.csv'
merged_df.to_csv(csv_path, index=False, encoding='utf-8')

# 결과를 XLSX 파일로 저장
xlsx_path = '../data/imdb_crew_cast.xlsx'
merged_df.to_excel(xlsx_path, index=False)

----

## 오류
- 이 코드로 데이터를 수집할 시 감독과 작가의 정보가 이상하게 크롤링 됨을 확인.
- 그 이유는 IMDb cast&crew 칸이 고정이 아니라 변동되기 때문에 감독 칸이 비어있을 시 작가칸을 첫 번째로 인식하고 그 데이터가 감독의 정보로 들어감